**In case of problems or questions, please first check the list of [Frequently Asked Questions (FAQ)](https://stardist.net/docs/faq.html).**

Please shutdown all other training/prediction notebooks before running this notebook (as those might occupy the GPU memory otherwise).

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D

np.random.seed(6)
lbl_cmap = random_label_cmap()

# Data

We assume that data has already been downloaded in via notebook [1_data.ipynb](1_data.ipynb).  
We now load images from the sub-folder `test` that have not been used during training.

In [ ]:
from cellpose import io, transforms
import skimage.io
basedir = '/home/kcutler/DataDrive/skelpose_test'
mask_filter = '_masks'
basedir = '/home/kcutler/DataDrive/ecVSsp/'
phase = Path(basedir).rglob('*c1.tif')
pfiles = [str(p) for p in phase]
imgs = [transforms.normalize99(skimage.io.imread(f),skel=True) for f in pfiles]
X = imgs.copy()

n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))

In [ ]:
# show all test images
if True:
    fig, ax = plt.subplots(7,8, figsize=(16,16))
    for i,(a,x) in enumerate(zip(ax.flat, X)):
        a.imshow(x if x.ndim==2 else x[...,0], cmap='gray')
        a.set_title(i)
    [a.axis('off') for a in ax.flat]
    plt.tight_layout()
None;

In [ ]:
len(imgs)

# Load trained model

If you trained your own StarDist model (and optimized its thresholds) via notebook [2_training.ipynb](2_training.ipynb), then please set `demo_model = False` below.

In [ ]:
demo_model = False

if demo_model:
    print (
        "NOTE: This is loading a previously trained demo model!\n"
        "      Please set the variable 'demo_model = False' to load your own trained model.",
        file=sys.stderr, flush=True
    )
    model = StarDist2D.from_pretrained('2D_demo')
else:
    model = StarDist2D(None, name='stardist', basedir='models')
None;

## Prediction

Make sure to normalize the input image beforehand or supply a `normalizer` to the prediction function.

Calling `model.predict_instances` will
- predict object probabilities and star-convex polygon distances (see `model.predict` if you want those)
- perform non-maximum suppression (with overlap threshold `nms_thresh`) for polygons above object probability threshold `prob_thresh`.
- render all remaining polygon instances in a label image
- return the label instances image and also the details (coordinates, etc.) of all remaining polygons

In [ ]:
img = normalize(X[16], 1,99.8, axis=axis_norm)

labels, details = model.predict_instances(img,prob_thresh=0.05,nms_thresh=0.7)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(img if img.ndim==2 else img[...,0], clim=(0,1), cmap='gray')
plt.imshow(labels, cmap=lbl_cmap, alpha=0.5)
plt.axis('off');

## Save predictions

Uncomment the lines in the following cell if you want to save the example image and the predictions to disk.  
See [this notebook](../other2D/export_imagej_rois.ipynb) for more details on how to export ImageJ ROIs.

In [ ]:
# save_tiff_imagej_compatible('example_image.tif', img, axes='YX')
# save_tiff_imagej_compatible('example_labels.tif', labels, axes='YX')
# export_imagej_rois('example_rois.zip', details['coord'])

# Example results

In [ ]:
def example(model, i, show_dist=True):
    img = normalize(X[i], 1,99.8, axis=axis_norm)
    labels, details = model.predict_instances(img)

    plt.figure(figsize=(13,10))
    img_show = img if img.ndim==2 else img[...,0]
    coord, points, prob = details['coord'], details['points'], details['prob']
    plt.subplot(121); plt.imshow(img_show, cmap='gray'); plt.axis('off')
    a = plt.axis()
    _draw_polygons(coord, points, prob, show_dist=show_dist)
    plt.axis(a)
    plt.subplot(122); plt.imshow(img_show, cmap='gray'); plt.axis('off')
    plt.imshow(labels, cmap=lbl_cmap, alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
example(model, 42)

In [ ]:
example(model, 1)

In [ ]:
example(model, 15, False)

# Model from paper

In [ ]:
model_paper = StarDist2D.from_pretrained('2D_paper_dsb2018')

In [ ]:
example(model_paper, 29)

# Versatile model

Try this model first if you have images that look similar to the training data in this example.

In [ ]:
model_versatile = StarDist2D.from_pretrained('2D_versatile_fluo')

In [ ]:
example(model_versatile, 30, False)

Show all available pretrained models:

In [ ]:
StarDist2D.from_pretrained()

In [ ]:
masks = [[]]*len(imgs)
for i in range(len(X)):
    img = normalize(X[i], 1,99.8, axis=axis_norm)
    masks[i], details = model.predict_instances(img)

In [ ]:
basedir = '/home/kcutler/DataDrive/skelpose_paper/Figure 6/StarDist'
io.check_dir(basedir)
import os
from pathlib import Path
if not os.path.isdir(basedir):
    os.mkdir(basedir)
# for j in range(n):
img_names = pfiles
for j in range(len(X)):
    savedir = basedir
    name = os.path.splitext(Path(img_names[j]).name)[0]
#     print(name)
    io.imsave(os.path.join(basedir,name+'_stardist_masks.tif'), masks[j])
#     io.save_masks(imgs, masks[j], flows[j], img_names,save_outlines=True,savedir=savedir)

In [ ]:
# save_masks(X, masks, [], file_names):
pfiles

In [ ]:
from pathlib import Path
imgdir = img_names[0]
prefix = Path(imgdir).parent.absolute()
name = imgdir[0:len(prefix)]
name

In [ ]:
prefix.__dict__

In [ ]:
prefix.

In [ ]:
img.shape

In [ ]:
base = '/home/kcutler/DataDrive/skelpose_paper/Figure 1/mx_raw_im'
base = '/home/kcutler/DataDrive/skelpose_paper/Figure 1/bt_raw_im'
img = io.imread(base+'.png')
print(img.shape)

img = normalize(img, 1,99.8, axis=axis_norm)
labels, details = model.predict_instances(img)

In [ ]:
plt.imshow(labels)

In [ ]:
io.imsave(base+'_stardist_masks.tif',labels.astype('int64')) 

In [ ]:
from cellpose.utils import outline_view, rescale
img0 = np.stack((img,img,img),axis=-1)

outlines = outline_view((rescale(img0)*255).astype('uint8'),labels)

import cv2
plt.imshow(outlines)
pic = outlines
io.imsave(base+'_stardist_outlines_big.png', cv2.resize((pic).astype(np.uint8), dsize=(10*pic.shape[1],10*pic.shape[0]), interpolation=cv2.INTER_NEAREST))
io.imsave(base+'_stardist_outlines.png',pic)

from skimage.segmentation import find_boundaries
boundaries = find_boundaries(labels,mode='inner')
io.imsave(base+'_stardist_boundaries.png', boundaries.astype('uint8'))